In [1]:
from cStringIO import StringIO

import fastavro

In [2]:
import pyspark

In [3]:
sc = pyspark.SparkContext()

URL normalization
---------------------

In [6]:
pickled_page_views = sc.pickleFile("../SuperWebAnalytics/master/pickle/page_view/")

In [7]:
pickled_page_views.count()

1409422

In [4]:
# raw_page_views = sc.binaryFiles("../SuperWebAnalytics/master/page_view/*")

In [11]:
# import fastavro
# from cStringIO import StringIO
# avro_page_views = raw_page_views.map(lambda t: StringIO(t[1]))
# json_page_views = avro_page_views.flatMap(fastavro.reader)

In [28]:
# page_views = raw_page_views.flatMap(lambda x: x[1].split("\n")).filter(len)#.values().map(eval)

In [8]:
from requests.utils import urlparse
def normalize_url(datum):
    pr = urlparse(datum['dataunit']['page_view']['page']['url'])
    datum['dataunit']['page_view']['page']['url'] = pr.hostname + pr.path
    return datum

In [9]:
normalized_page_views = pickled_page_views.map(normalize_url)

In [10]:
normalized_page_views.first()

{u'dataunit': {u'page_view': {u'nonce': 127688090,
   u'page': {u'url': u'mysite.com/blog'},
   u'person': {u'cookie': u'FGHIJ'}}},
 u'pedigree': {u'true_as_of_secs': 1438627996}}

Deduplicate pageviews
---------------------

In [11]:
import simplejson as json
distinct_normalized_page_views = normalized_page_views.map(json.dumps).distinct().map(json.loads)

In [12]:
distinct_normalized_page_views.first()

{'dataunit': {'page_view': {'nonce': 4028254285,
   'page': {'url': 'mysite.com/blog'},
   'person': {'cookie': 'PQRST'}}},
 'pedigree': {'true_as_of_secs': 1438913491}}

Pageviews over time
---------------------

In [13]:
granularity = {'h': 60 * 60}
granularity['d'] = granularity['h'] * 24
granularity['w'] = granularity['d'] * 7
granularity['m'] = granularity['w'] * 4
granularity

{'d': 86400, 'h': 3600, 'm': 2419200, 'w': 604800}

In [54]:
int(1438913491 / (60*60)) * (60*60)

1438912800

In [55]:
1438913491 - 1438913491 % (60*60)

1438912800

In [14]:
def hourly(datum):
    url = datum['dataunit']['page_view']['page']['url']
    true_as_of_secs = datum['pedigree']['true_as_of_secs']
    return (url, 'h', true_as_of_secs - true_as_of_secs % granularity['h']), datum

In [15]:
hourly_rollup = distinct_normalized_page_views.map(hourly).countByKey()

In [16]:
i, c = zip(*hourly_rollup.items())

In [17]:
import pandas as pd

In [19]:
pd.DataFrame({'count': c}, index=pd.MultiIndex.from_tuples(i, names=('url', 'hour'))).reset_index()

,url,hour,count
0,mysite.com/,1438912800,300054
1,mysite.com/blog,1438624800,101732
2,mysite.com/blog,1438912800,899778
3,mysite.com/,1438624800,101212
